In [87]:
!pip install tweepy markovify

In [88]:
import json
import tweepy

f = open('new.txt', 'rb')
new_file = f.readlines()


In [89]:
CONSUMER_API_KEY = new_file[0].strip()
CONSUMER_API_SECRET_KEY = new_file[1].strip()
ACCESS_TOKEN = new_file[2].strip()
ACCESS_TOKEN_SECRET = new_file[3].strip()


In [90]:
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [91]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

We select an user we would try to imitate

In [92]:
user_id = "jeremyphoward"

# lets collect users lates count=200 tweets

count = 200
user_tweets = api.user_timeline(screen_name=user_id, count=count, tweet_mode="extended")

In [93]:
user_tweets[:5]

[Status(_api=<tweepy.api.API object at 0x7fd4f4aa28d0>, _json={'created_at': 'Sun May 09 22:09:19 +0000 2021', 'id': 1391515652888829955, 'id_str': '1391515652888829955', 'full_text': 'RT @ylecun: Barlow Twins: a new super-simple self-supervised method to train joint-embedding architectures (aka Siamese nets) non contrasti…', 'truncated': False, 'display_text_range': [0, 140], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'ylecun', 'name': 'Yann LeCun', 'id': 48008938, 'id_str': '48008938', 'indices': [3, 10]}], 'urls': []}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 175282603, 'id_str': '175282603', 'name': 'Jeremy Howard', 'screen_name': 'jeremyphoward', 'location': 'Brisbane/Queensland, Australia', 'description': '🇦🇺 Distinguished resear

lets collect all of the tweets into one large corpus

In [94]:
tweet_corpus = []

for tweet in user_tweets:
  tweet_corpus.append(tweet.full_text)
tweets_text = ". ".join(tweet_corpus)

Lets proceed to process the text.We define a function that will replace any found instance of a URL with a new URL

In [95]:
import re

def replace_URLs(string, new_URL):
  """replace all URLs with a custom URL"""
  modified_string = re.sub(
      "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
      " " + new_URL + " ",
      string,
  )

  return modified_string

create a non standardlink and insert into the tweets. In ourcase, we used a URL shortner to obfuscate the fact that the link takes a user to google.com

In [96]:
mod_link = "https://urlzs.com/u8ZB"
processed_tweets_text = replace_URLs(tweets_text, mod_link)

In [97]:
processed_tweets_text

'RT @ylecun: Barlow Twins: a new super-simple self-supervised method to train joint-embedding architectures (aka Siamese nets) non contrasti…. @realcyberustadh @ylecun As a less-new research student, I feel the same way :). @wightmanr @ykilcher @calabi_and_yau Just to clarify - are you referring to SE as "attention like"?. @wightmanr @ykilcher @calabi_and_yau Very interesting - thanks Ross. RT @omarsar0: Web development feels like a super power.\n\nThis became even more obvious once I started to do machine learning engineering.…. @ykilcher @calabi_and_yau Are you sure it outperforms resnet? If it\'s just comparing the meaningless FLOPs metric then that tells us nothing. AFAIK nothing outperforms a modern resnet by quite some margin in practice.. @abhi1thakur FYI, you get something similar to this (but harder) with the imagewang dataset\n https://urlzs.com/u8ZB  RT @RachelTobac: Huge heads up on PayPal Twitter Tip Jar. If you send a person a tip using PayPal, when the receiver opens up 

train a markov model on the processed text and generate tweets

In [98]:
import markovify
markov_model = markovify.Text(processed_tweets_text)

Generate the desired number of tweets that contains the modified link

In [99]:
num_new_tweets_desired = 5
num_new_tweets_so_far = 0
generated_tweets = []



In [100]:
mod_link, generated_tweets

('https://urlzs.com/u8ZB', [])

In [101]:
# print(markov_model.make_sentence(tries=1000))

In [102]:
while num_new_tweets_so_far < num_new_tweets_desired:
  tweet = markov_model.make_short_sentence(140)
  if((mod_link in tweet) and (tweet not in generated_tweets)):
    generated_tweets.append(tweet)
    num_new_tweets_so_far += 1

I am generally a hawk about maintaining rules with a simple list of customers.
Still, @timnitGebru…. RT @PSampathkumarMD: I am generally a hawk about maintaining rules with a simple list of customers.
However,…. RT @chr1sa: It's now been almost a year ago we signed a @Salesforce Enterprise contract for @Heroku.
RT @TheZachMueller: Looks like a super power.
If we let impossible standar…. RT @feraldata: one of the most crucial scie…. RT @ylecun: Well, not *actually* conv free.
Be Quiet is FSP I think…. RT @amaarora: We asked and you answered!
Do you use pydantic or @tiangolo's FastAPI and/or want to fine-tune NVIDIA's Megatron models?
Proud to be part of computer science.
Outdoor masking has notable costs and re…. RT @zeynep: The WHO just updated its page on how COVID-19 transmits.
Around 2009,…. @rjurney @__mharrison__ It's on the Liar dataset from 2017 for a demo article.
RT @omarsar0: Web development feels like a new @fastdotai study group is brewing.
I'm an external on a PhD defense 

In [103]:
generated_tweets

['https://urlzs.com/u8ZB RT @huggingface: Welcome @nvidia @NVIDIAAI to the old issue, I quite like follow-ups.',
 'See below: https://urlzs.com/u8ZB RT @ConserveChange: What the actual....',
 '💪😃 https://urlzs.com/u8ZB …. RT @Chris_Said: This is my second translation!',
 'Australia: lol https://urlzs.com/u8ZB RT @ConserveChange: What the actual....',
 'See below: https://urlzs.com/u8ZB RT @_ScottCondron: New blog post!']

Publishing the tweet. lets look at the user friends

In [104]:
user = api.get_user(user_id)
for friend in user.friends():
  print(friend.screen_name)

SergeyI49013776
neurosp1ke
lucidrains
ringo_ring
terronk
prudholu
VictorDominello
ian_oppermann
bindureddy
BarryOSullivan
andrewm4894
MaverickPramit
ItsAmitJena
gordo_stevenson
riccardo_grin
CalebCity
stevenpbrumby
miguelmalvarez
draecomino
AgDeasey


Lets send out tweets. 

In [105]:
api.update_status('Next few (3) tweets would be from a bot impersonating Jeremy Howard. @jeremyphoward')

Status(_api=<tweepy.api.API object at 0x7fd4f4aa28d0>, _json={'created_at': 'Mon May 10 02:49:22 +0000 2021', 'id': 1391586130030600195, 'id_str': '1391586130030600195', 'text': 'Next few (3) tweets would be from a bot impersonating Jeremy Howard. @jeremyphoward', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'jeremyphoward', 'name': 'Jeremy Howard', 'id': 175282603, 'id_str': '175282603', 'indices': [69, 83]}], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">markovify-twitter</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 893104100627275776, 'id_str': '893104100627275776', 'name': 'Ashwin Jayaprakash', 'screen_name': 'fanbyprinciple', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protect

In [106]:
for i in range(3):
  api.update_status(generated_tweets[i])

api.update_status('Well. Thats all.')

Status(_api=<tweepy.api.API object at 0x7fd4f4aa28d0>, _json={'created_at': 'Mon May 10 02:49:23 +0000 2021', 'id': 1391586133650276352, 'id_str': '1391586133650276352', 'text': 'Well. Thats all.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">markovify-twitter</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 893104100627275776, 'id_str': '893104100627275776', 'name': 'Ashwin Jayaprakash', 'screen_name': 'fanbyprinciple', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 14, 'friends_count': 202, 'listed_count': 0, 'created_at': 'Thu Aug 03 13:39:45 +0000 2017', 'favourites_count': 826, 'utc_offset': None, 'time_zone': Non